In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.tri as mtri
import matplotlib.path as mpath
import pandas as pd
import json

In [2]:
casie_plt_boundary_file = "../input_fault_data/casie21/Casie21-R2T-PlateBdy_medflt-surface-mask-notnull.csv"
casie_pt_boundary_boundary_file = "../input_fault_data/casie21/casie21_platebdy_boundary.geojson"

In [3]:
pts = pd.read_csv(casie_plt_boundary_file)

In [4]:
pts.head()

,lon,lat,depth
0,-128.741935,50.0,-4579.860352
1,-128.709677,50.0,-4835.104980
2,-128.677419,50.0,-5090.997559
3,-128.645161,50.0,-5347.662598
4,-128.612903,50.0,-5605.197754


In [5]:
triang = mtri.Triangulation(pts.lon.values, pts.lat.values)

In [6]:
triang.triangles[0]

array([11254,  9165,  6678], dtype=int32)

In [7]:
def _get_tri_pts(tri, pts):
    p1 = pts.loc[tri[0]]
    p2 = pts.loc[tri[1]]
    p3 = pts.loc[tri[2]]
    
    return [p1, p2, p3]

def tri_centroid(tri, pts=pts):
    p1, p2, p3 = _get_tri_pts(tri, pts)
    
    lon = np.mean([p1.lon, p2.lon, p3.lon])
    lat = np.mean([p1.lat, p2.lat, p3.lat])
    depth = np.mean([p1.depth, p2.depth, p3.depth])
    
    return lon, lat, depth
    
    

In [8]:
tri_centroid(triang.triangles[0])

(-125.63440860200001, 45.06790123456667, -5070.732259116667)

In [9]:
tri_centroids = [tri_centroid(tri) for tri in triang.triangles]

In [10]:
centroid_latlons = np.array([[t[0],t[1]] for t in tri_centroids])

In [11]:
with open(casie_pt_boundary_boundary_file) as f:
    pbj = json.load(f)

In [12]:
bounds = pbj['features'][0]['geometry']['coordinates'][0]

bound_path = mpath.Path(bounds)

In [13]:
pts_in_bounds = bound_path.contains_points(centroid_latlons)

In [14]:
def tri_to_feature(tri, pts=pts):
    p1, p2, p3 = _get_tri_pts(tri, pts)
    f = {'geometry': {
            'coordinates': [
                [[np.round(p1.lon, 5), np.round(p1.lat, 5), np.round(p1.depth, 1)],
                 [np.round(p2.lon, 5), np.round(p2.lat, 5), np.round(p2.depth, 1)],
                 [np.round(p3.lon, 5), np.round(p3.lat, 5), np.round(p3.depth, 1)],
                 [np.round(p1.lon, 5), np.round(p1.lat, 5), np.round(p1.depth, 1)]]
                ],
             'type': 'Polygon'},
        'properties': {},
        'type': 'Feature'
    }
    
    return f
         

In [15]:
tri_to_feature(triang.triangles[0])

{'geometry': {'coordinates': [[[-125.51613, 43.77116, -4605.2],
    [-125.51613, 45.0754, -5494.6],
    [-125.87097, 46.35714, -5112.4],
    [-125.51613, 43.77116, -4605.2]]],
  'type': 'Polygon'},
 'properties': {},
 'type': 'Feature'}

In [16]:
tris_in_boundary = [tri_to_feature(tri)
                    for i, tri in enumerate(triang.triangles)
                    if pts_in_bounds[i]
                   ]

In [17]:
tri_geojson = {
    'type': 'FeatureCollection',
    'name': 'casie21_platebdy_boundary',
    'crs': {'type': 'name',
    'properties': {'name': 'urn:ogc:def:crs:OGC:1.3:CRS84'}},
    'features': tris_in_boundary
}

In [18]:
with open(
    "../crescent_cfm_files/casie21_plate_boundary.geojson", 'w') as f:
    json.dump(tri_geojson, f)